<a href="https://colab.research.google.com/github/vicentcamison/idal_ia3/blob/main/4%20Aprendizaje%20reforzado/Sesion%203/DQN_v0_vanilla.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# DQN v0 vanilla

In [ ]:
import gym
import numpy as np
import os
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import matplotlib.pyplot as plt
from utils import discrete_input
import random
from collections import deque
import datetime

Disable GPU computation for local devices


In [ ]:
# os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

## Gym selection and basic configurations

In [ ]:
gym_name_list = [
    {
        'name': 'CartPole-v0',
        'goal': 180,
        'v_min': 0,
        'v_max': 210,
        'ep': 50
    },
    {
        'name': 'MountainCar-v0',
        'goal': -150, 
        'v_min': -210,
        'v_max': 0,
        'ep': 20
    },
    {
        'name': 'Blackjack-v0',
        'goal': 0.10,
        'v_min': -20,
        'v_max': 20,
        'ep': 1000
    }
]

env_i = 0 #@param {type:"slider", min:0, max:2, step:1}

save_model: bool = False # @param {type:"boolean"}
show_plots: bool = True # @param {type:"boolean"}
render_env: bool = False # @param {type:"boolean"}
seed = 42 # @param {type:"integer"}

max_steps_per_episode = 200 # @param {type:"integer"}

stopping_reward_criteria = gym_name_list[env_i]['goal']

gym_name = gym_name_list[env_i]['name']

env = gym.make(gym_name)  # Create the environment
env.seed(seed)

if isinstance(env.observation_space, gym.spaces.tuple.Tuple):
    env = gym.wrappers.TransformObservation(env, lambda obs: discrete_input(obs, env.observation_space))
    num_inputs = sum([x.n for x in env.observation_space])  # 4
else:
    num_inputs = env.observation_space.shape[0]  # 4
num_actions = env.action_space.n  # 2

## Algorithm hyper-parameters

In [ ]:
# Factor of the ema that displays that tracks the averaged rewards
ema_ratio = 0.01  # @param {type:"number"}

# Ratio between generating experiences and sampling for training
training_ratio: int = 4 # @param  {type:"integer"}

# Size of the batch when sampling experiences
batch_size: int = 32 # @param {type:"integer"}

# Size of the buffer that stores the experiences
mem_length: int = 2048 # @param {type:"integer"}

# Discount factor for estimaing the futures rewards
gamma: float = 0.99  # @param {type:"number"}

# Initial and last probability for choosing exploration instead of explotation
epsilon: float = 1.0 # @param {type:"number"}
epsilon_min: float = 0.05 # @param {type:"number"}

# This is an estimation of the training iterations to tune the epsilon decay
approx_iterations: float = 1e6 # @param {type:"number"}

# The epsilon_decay reduce the exploration probability after each iteration
epsilon_decay: float = (epsilon_min / epsilon) ** (1 / approx_iterations)

# Factor of the ema that controls the updating weights of the target network
tau: float = 0.125 # @param {type:"number"}

# The usual factor that controls the amount of change the weights are updated
learning_rate = 0.01 # @param {type:"number"}

# For enabling the double dqn learning when choosing next Q-values
double_dqn_learning: bool = False # @param {type:"boolean"}

# Factor to define heuristically the size of the hidden layer.
hidden_size_factor = 16 # @param {type:"integer"}
num_hidden = num_inputs * num_actions * hidden_size_factor

## Load DQN models as Q-table approximators
We start with double DQN, where:
*   q_model estimates the Q-values used for action selection.
*   t_model is responsible for estimating the target Q values on training.

In [ ]:
q_model = keras.Sequential(layers=[layers.Input(shape=(num_inputs,)),
                                   layers.Dense(num_hidden, activation="relu"),
                                   layers.Dense(num_actions)],
                           name="q_model")

t_model = keras.Sequential(layers=[layers.Input(shape=(num_inputs,)),
                                   layers.Dense(num_hidden, activation="relu"),
                                   layers.Dense(num_actions)],
                           name="t_model")

optimizer = keras.optimizers.Adam(learning_rate=learning_rate)
# loss_function = keras.losses.Huber()
loss_function = keras.losses.MeanSquaredError()

## Tensorboard configuration

In [ ]:
implementation = "DQN_v0"
current_time = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
train_log_dir = os.path.join("./logs", gym_name, implementation, "T_" + current_time)
summary_writer = tf.summary.create_file_writer(train_log_dir)

In [ ]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

# from tensorboard import notebook
# notebook.list() # View open TensorBoard instances

# # Control TensorBoard display. If no port is provided, 
# # the most recently launched TensorBoard is used
# notebook.display(port=6006, height=1000) 

%tensorboard --logdir ./logs

## Main learning loop
It consist in main two steps:

1.   Collect experiences from the environment in episodes, with exploitation/exploration trade-off
2.   Sample past experiences to train the model using the Bellman equation.

Other sections are the mean reward tracking, stop-learning trigger, display status.


In [ ]:
memory = deque(maxlen=mem_length)
running_reward = None
episode_count = 0
epoch = 0
historic_reward = []
while True:  # Run until solved

    state = env.reset()
    state = tf.convert_to_tensor(state)
    state = tf.expand_dims(state, 0)
    episode_reward = 0
    for time_step in range(1, max_steps_per_episode):
        # env.render(); Adding this line would show the attempts
        # of the agent in a pop up window.

        epsilon *= epsilon_decay
        epsilon = max(epsilon_min, epsilon)
        if np.random.random() < epsilon:
            action = env.action_space.sample()
        else:
            action = np.argmax(q_model(state, training=False))

        # Apply the sampled action in our environment
        next_state, reward, done, _ = env.step(action)

        next_state = tf.convert_to_tensor(next_state)
        next_state = tf.expand_dims(next_state, 0)

        memory.append([state, action, reward, next_state, done])
        episode_reward += reward

        # ##### TRAIN MODEL ###############
        if len(memory) >= 2 * batch_size and time_step % training_ratio == 0:
            samples = random.sample(memory, batch_size)

            for sample in samples:
                state_i, action_i, reward_i, next_state_i, done_i = sample
                with tf.GradientTape() as tape:
                    q_state = q_model(state_i)[0, action_i]
                    if done_i:
                        q_target = reward_i
                    else:
                        if double_dqn_learning:
                            next_action_i = np.argmax(q_model(next_state_i, training=False))
                            q_next_state = t_model(next_state_i)[0, next_action_i]
                        else:
                            q_next_state = max(t_model(next_state_i)[0])
                        q_target = reward_i + q_next_state * gamma

                        loss = loss_function(tf.expand_dims(q_state, 0), tf.expand_dims(q_target, 0))

                        # Backpropagation
                        grads = tape.gradient(loss, q_model.trainable_variables)
                        optimizer.apply_gradients(zip(grads, q_model.trainable_variables))

            # Transfer weights to target model
            weights = q_model.get_weights()
            target_weights = t_model.get_weights()
            for j in range(len(target_weights)):
                target_weights[j] = weights[j] * tau + target_weights[j] * (1 - tau)
            t_model.set_weights(target_weights)
            # ###############################################

            if running_reward is not None:
                with summary_writer.as_default():
                    tf.summary.scalar('loss', loss, step=epoch)
                    tf.summary.scalar('ema_reward', running_reward, step=epoch)
                    tf.summary.scalar('epsilon', epsilon, step=epoch)
                epoch += 1
        state = next_state

        if done:
            break

    if running_reward is None:
        running_reward = episode_reward

    # Update running reward to check condition for solving
    running_reward = ema_ratio * episode_reward + (1 - ema_ratio) * running_reward
    historic_reward.append(running_reward)

    # Log details
    episode_count += 1
    if episode_count % gym_name_list[env_i]['ep'] == 0 and 'loss' in locals():
        template = "running reward: {:.2f} at episode {} with epsilon {:.2f} and loss {:.2f}"
        print(template.format(running_reward, episode_count, epsilon, loss))

    if running_reward > stopping_reward_criteria:  # Condition to consider the task solved
        print("Solved at episode {}!".format(episode_count))
        break

    if show_plots and episode_count % 10000000 == 0:
        plt.plot(historic_reward)
        plt.show()

running reward: 22.32 at episode 50 with epsilon 0.97 and loss 0.02
running reward: 21.91 at episode 100 with epsilon 0.94 and loss 0.75
running reward: 23.12 at episode 150 with epsilon 0.90 and loss 26.04
running reward: 22.16 at episode 200 with epsilon 0.88 and loss 4329.10
running reward: 21.10 at episode 250 with epsilon 0.85 and loss 5999591.00
running reward: 20.20 at episode 300 with epsilon 0.83 and loss 34603044864.00
running reward: 20.61 at episode 350 with epsilon 0.80 and loss 4123799040.00
running reward: 19.83 at episode 400 with epsilon 0.78 and loss 263790870528.00
running reward: 18.97 at episode 450 with epsilon 0.76 and loss 671070617600.00


## Save model weights for a later use (optional).

In [ ]:
if save_model:
    model_folder = os.path.join("./models", gym_name, implementation, "Ep_" + str(episode_count).zfill(5), "model")
    if not os.path.exists(model_folder):
        os.makedirs(model_folder)
    q_model.save_weights(filepath=model_folder, save_format="tf")

## Play with the trained agent

In [ ]:
episodes = 100 # @param {type:"integer"}
agent_rewards = []
for env_i in range(episodes):
    state = env.reset()
    episode_reward = 0

    for time_step in range(1, max_steps_per_episode):
        if render_env and gym_name != 'Blackjack-v0':
            env.render()  # Show the attempts of the agent in a pop up window.

        state = tf.convert_to_tensor(state)
        state = tf.expand_dims(state, 0)

        action = np.argmax(q_model(state))

        # Apply the sampled action in our environment
        state, reward, done, _ = env.step(action)
        episode_reward += reward

        if done:
            break
    agent_rewards.append(episode_reward)
  
print(f"After 100 episodes the mean reward is {np.mean(agent_rewards)}")

if show_plots:
    num_bins = 50
    x = np.array(agent_rewards)
    fig, ax = plt.subplots()

    # the histogram of the data
    n, bins, patches = ax.hist(x, num_bins, density=1)

    ax.set_xlabel('Episode rewards')
    ax.set_ylabel('Probability density')
    ax.set_title(f'Mean {np.mean(x).round(2)} +/- {np.std(x).round(2)}')

    # Tweak spacing to prevent clipping of ylabel
    fig.tight_layout()
    plt.show()

print("End of script!")